In [25]:
MODELS = [
    "openai/gpt-4.1",
    "openai/gpt-4.1-mini",
    "openai/gpt-4.1-nano",
    "openai/gpt-4o-mini",
    "openai/gpt-4o-2024-11-20",
    "deepseek/deepseek-chat-v3-0324",
    "deepseek/deepseek-r1-250528",
    "google/gemini-2.5-flash",
    "google/gemini-2.5-pro",
    "meta-llama/llama-3.3-70b-instruct",
    "meta-llama/llama-3.1-8b-instruct",
    "meta-llama/llama-3.2-3b-instruct",
    "meta-llama/llama-3.2-1b-instruct",
    "meta-llama/llama-4-maverick",
    "meta-llama/llama-4-scout",
    "qwen/qwen3-30b-a3b",
    "qwen/qwen3-32b",
    "qwen/qwen3-8b",
    "qwen/qwen3-1.7b",
    "baidu/ernie-4.5-turbo-128k",
    "baidu/ernie-4.5-0.3b",
    "baidu/ernie-4.5-21b-a3b"
]
SET_NAME = [
    "test",
    "renamed"
]

In [ ]:
from io import StringIO
import re
from contextlib import redirect_stdout
import pickle
import pandas
from tqdm.notebook import trange, tqdm

eval_data = {}
pbar_models = trange(len(MODELS)*len(SET_NAME), desc="Evaluation")

for model_name in MODELS:
    for set_name in SET_NAME:
        # load outputs from Llama-3.2-1B-Instruct.pkl
        with open(f"drive/MyDrive/lab/{set_name}/{model_name}-{set_name}.pkl", "rb") as f:
            outputs = pickle.load(f)

        assert(len(outputs) == 164)
        eval_data[f"{model_name}-{set_name}"] = []

        for sample in tqdm(outputs, desc=f"{model_name}-{set_name}", leave=False):
            prompt = sample["prompt"]
            code = sample["response"]
            # strip other text from code
            pattern = r"```.*"
            if re.match(pattern, code):
                code = re.split(pattern, code)[1]
            test_code = sample["test"]
            executable = "{}\n{}\n{}\ncheck({})".format(prompt, code, test_code, sample["entry_point"])

            try:
                exec_namespace = {}
                f = StringIO()
                with redirect_stdout(f):
                    exec(executable, exec_namespace)
                eval_data[f"{model_name}-{set_name}"].append(True)
            except Exception:
                eval_data[f"{model_name}-{set_name}"].append(False)

df = pandas.DataFrame(eval_data)
df.to_csv(f"lab/code_check.csv", index=False)
df

In [46]:
# Utils for viewing csv
import pandas as pd
import pickle

dir = "lab-legacy/gpt-4.1-nano-test (1).pkl"

with open(dir, "rb") as f:
    outputs = pickle.load(f)

df = pd.DataFrame(outputs)

df.to_csv("lab-legacy/test.csv")

df

,task_id,prompt,canonical_solution,test,entry_point,response
0,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,"for idx, elem in enumerate(numbers):\n ...","\n\nMETADATA = {\n 'author': 'jt',\n 'da...",has_close_elements,for i in range(len(numbers)):\n for...
1,HumanEval/1,from typing import List\n\n\ndef separate_pare...,result = []\n current_string = []\n ...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",separate_paren_groups,result = []\n current = ''\n depth =...
2,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",truncate_number,integer_part = int(number)\n decimal_pa...
3,HumanEval/3,from typing import List\n\n\ndef below_zero(op...,balance = 0\n\n for op in operations:\n...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",below_zero,balance = 0\n for operation in operatio...
4,HumanEval/4,from typing import List\n\n\ndef mean_absolute...,mean = sum(numbers) / len(numbers)\n re...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",mean_absolute_deviation,if not numbers:\n return 0.0\n m...
...,...,...,...,...,...,...
159,HumanEval/159,"\ndef eat(number, need, remaining):\n """"""\n...",if(need <= remaining):\n return [ n...,def check(candidate):\n\n # Check some simp...,eat,total_eaten = number + need\n if remain...
160,HumanEval/160,"\ndef do_algebra(operator, operand):\n """"""\...",expression = str(operand[0])\n for oprt...,def check(candidate):\n\n # Check some simp...,do_algebra,"expression = str(operand[0])\n for op, ..."
161,HumanEval/161,"\ndef solve(s):\n """"""You are given a string...",flg = 0\n idx = 0\n new_str = list(s...,def check(candidate):\n\n # Check some simp...,solve,result = []\n has_letter = False\n\n ...
162,HumanEval/162,"\ndef string_to_md5(text):\n """"""\n Given...",import hashlib\n return hashlib.md5(tex...,def check(candidate):\n\n # Check some simp...,string_to_md5,"import hashlib\n\n if text == """":\n ..."
